# LightGBM

O LightGBM é um tipo de algoritmo usado para modelos de Aprendizagem de Máquina (AM), que utiliza várias árvores para fazer suas previsões, além de dar um "boosting", uma impulsionada para o código.

Cada vez que o modelo adiciona uma nova árvore, ela corrige a anterior, tornando seu modelo mais robusto.

<center>

![image.png](../figures/light_gbm.png)

</center>

In [17]:
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


## Lendo o Dataset tratado, transformando-o em um DataFrame e armazenando nas variáveis, as colunas referentes as suas características e rótulos

In [18]:
df = pd.read_csv('dataset_tratado.csv')

x = df.drop('ZSN', axis=1)  # Armazenando todas as colunas de características, exceto a última pois ela é a coluna de rótulo
y = df['ZSN']               # Armazenado a última coluna, que será o nosso rótulo

## Treinando o modelo, usando 20% dos dados para teste e o restante para treino

In [19]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

train_data = lgb.Dataset(x_train, label=y_train)
test_data = lgb.Dataset(x_test, label=y_test, reference=train_data)

## Pegando os parâmetros usandos e classificando-os

Parte que ajusta o comportamento do modelo com base na sua personalização, dessa forma, isso torna o modelo mais eficiente de acordo com o que se espera dele

In [20]:
params = {
    'objective': 'binary',      # Define o objetivo da tarefa de aprendizado (regressão ou clasificação)
    'metric': 'binary_error',   # Parte que avalia o desempenho do modelo durante o treinamento, calcula a qualidade das previsões de cada iteração
    'boosting_type': 'gbdt',    # Define o tipo de boosting a ser usado (gbdt, rf, dart, goss)
    'num_leaves': 35,           # Número máximo de folhas em uma árvore
    'learning_rate': 0.05,      # Taxa de aprendizado que contrala a contribuição de cada árvore no modelo
    'feature_fraction': 0.9     # Fração de featurs a serem consideradas em cada iteração para treinar a árvore
}

## Parte dedicada para as métricas do código

In [21]:
num_round = 100
bst = lgb.train(params, train_data, num_round, valid_sets=[test_data])

y_pred = bst.predict(x_test, num_iteration=bst.best_iteration)
y_pred_binary = [1 if pred > 0.5 else 0 for pred in y_pred]

print(classification_report(y_test, y_pred_binary, zero_division=0))

[LightGBM] [Info] Number of positive: 314, number of negative: 1046
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000757 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 504
[LightGBM] [Info] Number of data points in the train set: 1360, number of used features: 70
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.230882 -> initscore=-1.203336
[LightGBM] [Info] Start training from score -1.203336
              precision    recall  f1-score   support

           0       0.81      0.98      0.89       260
           1       0.80      0.25      0.38        80

    accuracy                           0.81       340
   macro avg       0.80      0.62      0.63       340
weighted avg       0.81      0.81      0.77       340

